In [2]:
import environment
import agent
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from time import sleep
from util import score_func
import random
import torch

random.seed(77)
train = True
render = True
masking = True
avg_score = 0
max_score = 0
epoch = 50000 if train else 1

if masking :
    save_interval = 100
else :
    save_interval = 500

agent = agent.DQNAgent(state_size=44, action_size=11, train=train, state_dict=None)
# agent = agent.DQNAgent(state_size=44, action_size=11, train=train, state_dict=torch.load("weight/save_800_s0_lr0.00025.pth"))
if train:
    score_history = []
    p_history = []
    q_history = []
    s_history = []
    c_history = []
    step_history = []



for i in tqdm(range(epoch)):
    env = environment.FactoryEnv()
    
    
    submission = pd.read_csv("data/sample_submission.csv")
    actions = []
    process_n = []   

    if render:
          if i % save_interval == save_interval-1:
                render_p_history = [] 
                render_q_history = [] 
                render_s_history = [] 
                render_c_history = [] 
                render_reward_history = []
                render_score_history = []

    for step in range(2184):       
        state = env.get_state()
        env.get_mask()
        action = agent.get_action(state, env.mask, masking)
        
        # 제출파일 만들기
        action_name = env.action_map[action]
        action_process = 0
        if 'PROCESS' in action_name:
            mylist = action_name.split("_")
            action_name = mylist[0]
            action_process = float(mylist[1])                
        if 'CHANGE' in action_name:
            action_name = "CHANGE_" + str(env.prev_change+1) + str(action-3)
        actions.append(action_name)
        process_n.append(action_process)
        
        next_state, reward, done = env.step(action, train, masking)
      
        if render:
            if i % save_interval == save_interval-1:
                render_score_history.append(env.prev_score)
                render_p_history.append(score_func(env.p, 32550830) * 100)
                render_q_history.append(score_func(env.q, 32550830) * 100)
                render_s_history.append(score_func(env.s_t, env.step_count) / (1 + 0.1*env.s_n) * 100)
                render_c_history.append(score_func(env.c_t, env.step_count) / (1 + 0.1*env.c_n) * 100)
                render_reward_history.append(reward)

        if train: 
            agent.append_sample(state, action, reward, next_state, done)
            agent.train_model()

        if done:
            break
    
    avg_step = 0
        
#     if not train:
#         aaa = list()
#         bbb = list()
#         for j in range(4):
#             env.J[j].save_graph(f"performance/performance{j}.png")
#             info = np.array(env.J[j].info)
#             t, w, o = info[:, 0], info[:, 1], info[:, 2]
#             aaa.append(sum(w)/t[-1])
#             bbb.append(sum(o)/t[-1])
#         print(sum(aaa)/len(aaa))
#         print(sum(bbb)/len(bbb))
    
    if train:
        avg_step = env.step_count if avg_step == 0 else avg_step * 0.9 + env.step_count * 0.1
        score_history.append(env.prev_score)
        p_history.append(score_func(env.p, 32550830) * 100) 
        q_history.append(score_func(env.q, 32550830) * 100) 
        s_history.append(score_func(env.s_t, 2184) / (1 + 0.1*env.s_n) * 100) 
        c_history.append(score_func(env.c_t, 2184) / (1 + 0.1*env.c_n) * 100) 
        step_history.append(env.step_count)
        if i % save_interval == save_interval-1:
                
            
            # 그래프 그리기

            if masking == False :
                torch.save(agent.model.state_dict(), f"weight/masking_{i + 1}_s{avg_step}_lr{agent.learning_rate}.pth")
                plt.rcParams["figure.figsize"] = (25,6)
                fig = plt.figure()
                step = plt.scatter(x = range(len(step_history)), y = step_history, label = 'step', color = 'black', s=1)
                plt.savefig(f"graph/masking_{i+1}_lr{agent.learning_rate}_s{avg_step}.png")
                plt.clf()

            else :
                torch.save(agent.model.state_dict(), f"weight/save_{i + 1}_s{env.prev_score}_lr{agent.learning_rate}.pth")
                plt.rcParams["figure.figsize"] = (25,6)
                fig, ax1 = plt.subplots()
                over, = ax1.plot(p_history, label = 'over')
                under, = ax1.plot(q_history, label = 'under')
                stop, = ax1.plot(s_history, label = 'stop')
                change, = ax1.plot(c_history, label = 'change')
                score, = ax1.plot(score_history, label = 'score')
                ax1.tick_params(axis='y')
                ax2 = ax1.twinx()
                step, = ax2.plot(step_history, label = 'step', color = 'black')
                ax2.tick_params(axis='y', labelcolor='black')

                plt.legend([score, over, under, stop, change, step], ["score", "over", "under", "stop", "change", "step"])
                
                plt.savefig(f"graph/save_{i + 1}_s{env.prev_score}_lr{agent.learning_rate}.png")
                plt.clf()

                if render:
                    plt.rcParams["figure.figsize"] = (25,6)
                    fig, ax1 = plt.subplots()
                    score, = ax1.plot(render_score_history, label = 'score')
                    over, = ax1.plot(render_p_history, label = 'over')
                    under, = ax1.plot(render_q_history, label = 'under')
                    stop, = ax1.plot(render_s_history, label = 'stop')
                    change, = ax1.plot(render_c_history, label = 'change')
                    ax1.tick_params(axis='y')
                    ax2 = ax1.twinx()
                    reward, = ax2.plot(render_reward_history, label = 'step', color = 'black')
                    ax2.tick_params(axis='y', labelcolor='black')
                    plt.legend([score, over, under, stop, change, reward], ["score", "over", "under", "stop", "change", "reward"])
                    plt.savefig(f"graph/save_render_{i+1}_s{env.prev_score}.png")
                    plt.clf()

        if env.prev_score > max_score :
            max_score = env.prev_score
            torch.save(agent.model.state_dict(), f"weight/save_{i + 1}_s{env.prev_score}_lr{agent.learning_rate}.pth")
            
#             if render:
#                 plt.rcParams["figure.figsize"] = (25,6)
#                 fig, ax1 = plt.subplots()
#                 score, = ax1.plot(render_score_history, label = 'score')
#                 over, = ax1.plot(render_p_history, label = 'over')
#                 under, = ax1.plot(render_q_history, label = 'under')
#                 stop, = ax1.plot(render_s_history, label = 'stop')
#                 change, = ax1.plot(render_c_history, label = 'change')
#                 ax1.tick_params(axis='y')
#                 ax2 = ax1.twinx()
#                 reward, = ax2.plot(render_reward_history, label = 'step', color = 'black')
#                 ax2.tick_params(axis='y', labelcolor='black')
#                 plt.legend([score, over, under, stop, change, reward], ["score", "over", "under", "stop", "change", "reward"])
#                 plt.savefig(f"graph/save_render_{i+1}_s{env.prev_score}.png")
#                 plt.clf()
            
            submission.loc[:, "Event_A"] = actions
            submission.loc[:, "MOL_A"] = process_n
            submission.loc[:, "Event_B"] = actions
            submission.loc[:, "MOL_B"] = process_n

            PRTs = submission[["PRT_1", "PRT_2", "PRT_3", "PRT_4"]].values
            for i in (np.array(env.PRT) - np.array([0,0,554])):
                if i[2] >= 0:
                    PRTs[int(i[2]), int(i[0])] = i[1]
            PRTs = np.ceil(PRTs * 1.2)
            PRTs = PRTs.astype(int)
            submission.loc[:, "PRT_1":"PRT_4"] = PRTs
            submission.to_csv(f"submission/save_{env.prev_score}.csv", index=False)
            

  0%|                                                                                        | 0/50000 [00:00<?, ?it/s]

0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-1.8265068265068294
-0.008325008325005712
-0.008325008325009264
-0.008325008325009264
-0.008325008325005712
-0.008325008325009264
0.0
0.0
-1.5186202686202677
-0.007631257631256716
-0.007631257631256716
-0.007631257631260269
-0.007631257631256716
-0.007631257631256716
-0.007631257631256716
-0.007631257631260269
-0.007631257631256716
-0.007631257631256716
-0.007631257631256716
-0.007631257631260269
-0.007631257631256716
0.0
-1.277942143326758
-0.007044237813470744
-0.007044237813467191
-0.007044237813470744
-0.007044237813467191
-0.007044237813467191
-0.007044237813470744
-0.007044237813467191
-0.007044237813470744
-0.007044237813467191
-0.007044237813467191
-0.007044237813470744
-0.007044237813467191
0.0
0.0
-1.0893410618685344
-0.006541077969647091
-0.006541077969650644
-0.006541077969650644
-0.006541077969647091
-0.006541077969650644
0.0
0.0
-0.9419152276295151
-0.00610500

  0%|                                                                                        | 0/50000 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
if train == False:
    submission.loc[:, "Event_A"] = actions
    submission.loc[:, "MOL_A"] = process_n
    submission.loc[:, "Event_B"] = actions
    submission.loc[:, "MOL_B"] = process_n

    PRTs = submission[["PRT_1", "PRT_2", "PRT_3", "PRT_4"]].values
    for i in (np.array(env.PRT) - np.array([0,0,554])):
        if i[2] >= 0:
            PRTs[int(i[2]), int(i[0])] = i[1]
    PRTs = np.ceil(PRTs * 1.2)
    PRTs = PRTs.astype(int)
    submission.loc[:, "PRT_1":"PRT_4"] = PRTs
    submission.to_csv(f"submission/save_{env.prev_score}.csv", index=False)    

In [ ]:
order = pd.read_csv("data/order.csv")
stock = pd.read_csv("data/stock.csv")
cut_yield = pd.read_csv("data/cut_yield.csv")